## Memory 

Getting Started with Memory API Tutorial 🚀
Welcome! This interactive tutorial will guide you through using the Memory API, a powerful tool for document storage and retrieval. Whether you're new to vector databases or an experienced developer, this notebook will help you understand the basics and get up and running quickly.
What you'll learn:

How to set up and configure the Memory API client
Creating and managing memory banks (vector stores)
Different ways to insert documents into the system
How to perform intelligent queries on your documents

Prerequisites:

Basic Python knowledge
A running instance of the Memory API server (we'll use localhost in 
this tutorial)

Before you begin, please ensure Llama Stack is installed and set up by following the [Getting Started Guide](https://llama-stack.readthedocs.io/en/latest/getting_started/index.html).

Let's start by installing the required packages:

Set up your connection parameters:

In [1]:
HOST = "localhost"  # Replace with your host
PORT = 8321        # Replace with your port
MODEL_NAME='meta-llama/Llama-3.2-3B-Instruct'
MEMORY_BANK_ID="tutorial_bank"

In [2]:
# Install the client library and a helper package for colored output
#!pip install llama-stack-client termcolor

# 💡 Note: If you're running this in a new environment, you might need to restart
# your kernel after installation

1. **Initial Setup**

First, we'll import the necessary libraries and set up some helper functions. Let's break down what each import does:

llama_stack_client: Our main interface to the Memory API
base64: Helps us encode files for transmission
mimetypes: Determines file types automatically
termcolor: Makes our output prettier with colors

❓ Question: Why do we need to convert files to data URLs?
Answer: Data URLs allow us to embed file contents directly in our requests, making it easier to transmit files to the API without needing separate file uploads.

In [3]:
import base64
import json
import mimetypes
import os
from pathlib import Path

from llama_stack_client import LlamaStackClient
from llama_stack_client.types.memory_insert_params import Document
from termcolor import cprint

# Helper function to convert files to data URLs
def data_url_from_file(file_path: str) -> str:
    """Convert a file to a data URL for API transmission

    Args:
        file_path (str): Path to the file to convert

    Returns:
        str: Data URL containing the file's contents

    Example:
        >>> url = data_url_from_file('example.txt')
        >>> print(url[:30])  # Preview the start of the URL
        'data:text/plain;base64,SGVsbG8='
    """
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")

    with open(file_path, "rb") as file:
        file_content = file.read()

    base64_content = base64.b64encode(file_content).decode("utf-8")
    mime_type, _ = mimetypes.guess_type(file_path)

    data_url = f"data:{mime_type};base64,{base64_content}"
    return data_url

2. **Initialize Client and Create Memory Bank**

Now we'll set up our connection to the Memory API and create our first memory bank. A memory bank is like a specialized database that stores document embeddings for semantic search.
❓ Key Concepts:

embedding_model: The model used to convert text into vector representations
chunk_size: How large each piece of text should be when splitting documents
overlap_size: How much overlap between chunks (helps maintain context)

✨ Pro Tip: Choose your chunk size based on your use case. Smaller chunks (256-512 tokens) are better for precise retrieval, while larger chunks (1024+ tokens) maintain more context.

In [4]:
# Initialize client
client = LlamaStackClient(
    base_url=f"http://{HOST}:{PORT}",
)

# Let's see what providers are available
# Providers determine where and how your data is stored
providers = client.providers.list()
provider_id = providers["memory"][0].provider_id
print("Available providers:")
#print(json.dumps(providers, indent=2))
print(providers)
# Create a memory bank with optimized settings for general use
client.memory_banks.register(
    memory_bank_id=MEMORY_BANK_ID,
    params={
        "embedding_model": "all-MiniLM-L6-v2",
        "chunk_size_in_tokens": 512,
        "overlap_size_in_tokens": 64,
    },
    provider_id=provider_id,
)

Available providers:
{'inference': [ProviderInfo(provider_id='ollama', provider_type='remote::ollama')], 'memory': [ProviderInfo(provider_id='faiss', provider_type='inline::faiss')], 'safety': [ProviderInfo(provider_id='llama-guard', provider_type='inline::llama-guard')], 'agents': [ProviderInfo(provider_id='meta-reference', provider_type='inline::meta-reference')], 'telemetry': [ProviderInfo(provider_id='meta-reference', provider_type='inline::meta-reference')]}


3. **Insert Documents**
   
The Memory API supports multiple ways to add documents. We'll demonstrate two common approaches:

Loading documents from URLs
Loading documents from local files

❓ Important Concepts:

Each document needs a unique document_id
Metadata helps organize and filter documents later
The API automatically processes and chunks documents

In [5]:
# Example URLs to documentation
# 💡 Replace these with your own URLs or use the examples
urls = [
    "memory_optimizations.rst",
    "chat.rst",
    "llama3.rst",
]

# Create documents from URLs
# We add metadata to help organize our documents
url_documents = [
    Document(
        document_id=f"url-doc-{i}",  # Unique ID for each document
        content=f"https://raw.githubusercontent.com/pytorch/torchtune/main/docs/source/tutorials/{url}",
        mime_type="text/plain",
        metadata={"source": "url", "filename": url},  # Metadata helps with organization
    )
    for i, url in enumerate(urls)
]

# Example with local files
# 💡 Replace these with your actual files
local_files = ["example.txt", "readme.md"]
file_documents = [
    Document(
        document_id=f"file-doc-{i}",
        content=data_url_from_file(path),
        metadata={"source": "local", "filename": path},
    )
    for i, path in enumerate(local_files)
    if os.path.exists(path)
]

# Combine all documents
all_documents = url_documents + file_documents

# Insert documents into memory bank
response = client.memory.insert(
    bank_id= MEMORY_BANK_ID,
    documents=all_documents,
)

print("Documents inserted successfully!")

Documents inserted successfully!


4. **Query the Memory Bank**
   
Now for the exciting part - querying our documents! The Memory API uses semantic search to find relevant content based on meaning, not just keywords.
❓ Understanding Scores:

Generally, scores above 0.7 indicate strong relevance
Consider your use case when deciding on score thresholds

In [6]:
def print_query_results(query: str):
    """Helper function to print query results in a readable format

    Args:
        query (str): The search query to execute
    """
    print(f"\nQuery: {query}")
    print("-" * 50)
    response = client.memory.query(
        bank_id= MEMORY_BANK_ID,
        query=[query],  # The API accepts multiple queries at once!
    )

    for i, (chunk, score) in enumerate(zip(response.chunks, response.scores)):
        print(f"\nResult {i+1} (Score: {score:.3f})")
        print("=" * 40)
        print(chunk)
        print("=" * 40)

# Let's try some example queries
queries = [
    "How do I use LoRA?",  # Technical question
    "Tell me about memory optimizations",  # General topic
    "What are the key features of Llama 3?"  # Product-specific
]


for query in queries:
    print_query_results(query)


Query: How do I use LoRA?
--------------------------------------------------

Result 1 (Score: 1.166)
Chunk(content=".md>`_ to see how they differ.\n\n\n.. _glossary_peft:\n\nParameter Efficient Fine-Tuning (PEFT)\n--------------------------------------\n\n.. _glossary_lora:\n\nLow Rank Adaptation (LoRA)\n^^^^^^^^^^^^^^^^^^^^^^^^^^\n\n\n*What's going on here?*\n\nYou can read our tutorial on :ref:`finetuning Llama2 with LoRA<lora_finetune_label>` to understand how LoRA works, and how to use it.\nSimply stated, LoRA greatly reduces the number of trainable parameters, thus saving significant gradient and optimizer\nmemory during training.\n\n*Sounds great! How do I use it?*\n\nYou can finetune using any of our recipes with the ``lora_`` prefix, e.g. :ref:`lora_finetune_single_device<lora_finetune_recipe_label>`. These recipes utilize\nLoRA-enabled model builders, which we support for all our models, and also use the ``lora_`` prefix, e.g.\nthe :func:`torchtune.models.llama3.llama3` mode

Awesome, now we can embed all our notes with Llama-stack and ask it about the meaning of life :)

Next up, we will learn about the safety features and how to use them: [notebook link](./06_Safety101.ipynb).